In [2]:
import yfinance as yf
import pandas as pd

In [3]:
import ssl
import urllib.request

# Set the SSL certificate verification paths
ssl._create_default_https_context = ssl._create_unverified_context

# Make the request
response = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
# Handle the response
# ...

In [4]:
tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0].Symbol

In [5]:
tickers = tickers.to_list()

In [6]:
price_df = yf.download(tickers, start="2010-01-01")

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No timezone found, symbol may be delisted


In [7]:
def slice_df (symbol):
    sliced = price_df.copy()
    sliced = sliced[sliced.columns[sliced.columns.get_level_values(1) == symbol ]]
    sliced.columns = sliced.columns.droplevel(1)
    sliced.loc[:,"price"] = sliced.Open.shift(-1)
    return sliced

In [8]:
def applyindicators (df):
    df["SMA_200"] = df.Close.rolling(200).mean()
    df["SMA_20"] = df.Close.rolling(20).mean()
    df["stddev"]= df.Close.rolling(20).std()
    df["Upper"]= df.SMA_20 + 2 * df.stddev
    df["Lower"] = df.SMA_20 - 2 * df.stddev
    df ["rsi"] = ta.momentum.rsi(df.Close, 2)

In [9]:
def ma_cal (df, slow, fast):
    df["slow"] = df.Close.rolling(slow).mean()
    df["fast"] = df.Close.rolling(fast).mean()

In [27]:
def backtest(df, slow, fast):
    ma_cal (df, slow, fast)
    in_position = False
    profits = []
    
    for index, row in df.iterrows():
        if not in_position:
            if row.slow < row.fast:
                buyprice = row.price
                in_position = True
        if in_position:
                if row.slow > row.fast:
                    profit = (row.price - buyprice)/buyprice
                    profits.append(profit)
                    in_position = False
                    
    if in_position:
        profit =(row.Close - buyprice)/buyprice
        profits.append(profit)
    gain = (pd.Series(profits) + 1).prod()
    
    return gain

In [11]:

AAPL = slice_df("AAPL")

In [12]:
AAPL.head()

,Adj Close,Close,High,Low,Open,Volume,price
Date,,,,,,,
2010-01-04 00:00:00,6.505279,7.643214,7.660714,7.585000,7.622500,493729600,7.664286
2010-01-05 00:00:00,6.516528,7.656429,7.699643,7.616071,7.664286,601904800,7.656429
2010-01-06 00:00:00,6.412874,7.534643,7.686786,7.526786,7.656429,552160000,7.562500
2010-01-07 00:00:00,6.401015,7.520714,7.571429,7.466071,7.562500,477131200,7.510714
2010-01-08 00:00:00,6.443573,7.570714,7.571429,7.466429,7.510714,447610800,7.600000


In [28]:
backtest(AAPL, 100,50)

9.728320042662135

In [14]:
results = []

for sym in tickers:
    subdf = slice_df (sym)
    print("result for" + sym)
    print(backtest (subdf, 100,50))
    results.append(backtest (subdf, 100,50))
    

result forMMM
1.4823914512417231
result forAOS
3.685766204139639
result forABT
1.4378484404677188
result forABBV
1.5645863261134496
result forACN
2.4255573838723232
result forATVI
2.4599490770161974
result forADM
1.3738949353899947
result forADBE
5.259698492826578
result forADP
1.320237255653171
result forAAP
2.2492246926368455
result forAES
1.9034151147847516
result forAFL
2.701070451468886
result forA
2.3682717109492812
result forAPD
1.6832109409363807
result forAKAM
1.0413048718858373
result forALK
3.2438669894664094
result forALB
2.7479769680492288
result forARE
0.6044661713879131
result forALGN
4.84519215383065
result forALLE
0.6542165504002196
result forLNT
1.431672133172476
result forALL
2.29031695176343
result forGOOGL
2.5022949435811195
result forGOOG
2.625765908071999
result forMO
0.7679257207955459
result forAMZN
4.905698876447606
result forAMCR
0.5734304066700263
result forAMD
8.07362607638119
result forAEE
1.41883064971626
result forAAL
2.1702767892804284
result forAEP
0.9

C:\Temp\ipykernel_14112\3383643401.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gain = (pd.Series(profits) + 1).prod()


1.0
result forBBY
0.6414867371707058
result forBIO
4.332368354981748
result forTECH
2.8648079547326324
result forBIIB
3.1511951907236297
result forBLK
1.9176223543179498
result forBK
1.2830404918225704
result forBA
1.7457903986741343
result forBKNG
3.483549595001998
result forBWA
1.6463955131286137
result forBXP
0.5171163398012693
result forBSX
1.9443842048880546
result forBMY
1.016633332772107
result forAVGO
9.175474214312187
result forBR
1.796305602076165
result forBRO
3.3976495013682744
result forBF.B


C:\Temp\ipykernel_14112\3383643401.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gain = (pd.Series(profits) + 1).prod()


1.0
result forBG
1.2833572850247978
result forCHRW
0.7203606091010033
result forCDNS
2.7961580725226725
result forCZR
1.0532426473065881
result forCPT
1.5931131865104822
result forCPB
1.0566823211501126
result forCOF
1.8289587251881136
result forCAH
1.2415690524382843
result forKMX
0.7324219145000426
result forCCL
0.3491830319520323
result forCARR
1.5709464118006966
result forCTLT
0.7692719542331927
result forCAT
2.397835554688621
result forCBOE
2.0721011947538197
result forCBRE
2.743137034965472
result forCDW
3.125127052057056
result forCE
1.5686353244083306
result forCNC
4.490509862581629
result forCNP
1.867948136785186
result forCDAY
0.6276421937468134
result forCF
3.0486189678545585
result forCRL
3.3242238103986934
result forSCHW
2.403857483540488
result forCHTR
4.460560186369811
result forCVX
1.1263626351463476
result forCMG
4.538230689572796
result forCB
1.0553895458701497
result forCHD
2.819240678909155
result forCI
4.107714328864216
result forCINF
3.2089592599632333
result forC

C:\Temp\ipykernel_14112\3383643401.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gain = (pd.Series(profits) + 1).prod()


1.0
result forGEN
0.7131914270190091
result forGNRC
9.193902321341382
result forGD
3.193858789420668
result forGE
0.6067952834460489
result forGIS
1.1468341856337947
result forGM
1.2812581559992031
result forGPC
1.9038471410064768
result forGILD
4.325875014095781
result forGL
2.55782672940962
result forGPN
3.078263090573325
result forGS
1.4867831176552861
result forHAL
0.4458008676370769
result forHIG
3.3188752863891713
result forHAS
1.588618625645758
result forHCA
3.636696612450316
result forPEAK
0.6818818500827054
result forHSIC
1.4834077777171695
result forHSY
3.1892950944836826
result forHES
0.6980752418542308
result forHPE
0.9585015735988941
result forHLT
1.5179669847759982
result forHOLX
0.7334548745242011
result forHD
2.3876092794605834
result forHON
2.8533035918157226
result forHRL
2.6911917700036794
result forHST
0.9780801502244512
result forHWM
0.9832614603467773
result forHPQ
2.401382125249566
result forHUM
4.449272205423875
result forHBAN
1.2005495229282108
result forHII
3.

In [15]:
profits = pd.DataFrame( {"profit":results}, index = tickers)


In [16]:
profits.to_excel("profits.xlsx")

In [17]:
profits = profits.sort_values(by="profit",ascending= False)

In [18]:
profits.tail()

,profit
EIX,0.402236
CCL,0.349183
PCG,0.251056
OXY,0.169404
APA,0.050377


In [19]:
(slice_df("CCL").Close.pct_change()+1).prod()

0.29288833253592955

In [20]:
sololong = []

for i in tickers:
    longdf = slice_df (i)
    print("result for" + i)
    sololong.append((longdf.Close.pct_change()+1).prod())
    

result forMMM
result forAOS
result forABT
result forABBV
result forACN
result forATVI
result forADM
result forADBE
result forADP
result forAAP
result forAES
result forAFL
result forA
result forAPD
result forAKAM
result forALK
result forALB
result forARE
result forALGN
result forALLE
result forLNT
result forALL
result forGOOGL
result forGOOG
result forMO
result forAMZN
result forAMCR
result forAMD
result forAEE
result forAAL
result forAEP
result forAXP
result forAIG
result forAMT
result forAWK
result forAMP
result forABC
result forAME
result forAMGN
result forAPH
result forADI
result forANSS
result forAON
result forAPA
result forAAPL
result forAMAT
result forAPTV
result forACGL
result forANET
result forAJG
result forAIZ
result forT
result forATO
result forADSK
result forAZO
result forAVB
result forAVY
result forAXON
result forBKR
result forBALL
result forBAC
result forBBWI
result forBAX
result forBDX
result forWRB
result forBRK.B
result forBBY
result forBIO
result forTECH
result forBIIB

In [21]:
sololong

[1.2266923687763924,
 9.305985605097378,
 4.242648013660575,
 4.19589991719979,
 6.334680340698665,
 6.600884520254152,
 2.3584366965399997,
 9.054461719888522,
 5.685958113072546,
 2.9777116068160177,
 1.6444769141750328,
 2.8093336188141365,
 5.9796882884146445,
 3.783629971731198,
 2.9722222487949383,
 4.828219264911009,
 4.611446395191241,
 1.9126672621716303,
 16.43837758657106,
 2.2077145171185846,
 3.6112025402489816,
 3.7494244398284478,
 6.674770849967003,
 6.739807051607946,
 2.330975186156828,
 15.533980184229497,
 1.3259740637735773,
 8.928866217876054,
 3.2330691376083784,
 2.832285174612866,
 2.6170579861301637,
 3.6366081187053485,
 1.668785616817031,
 4.473078646841029,
 6.526269748953344,
 7.0717146805482685,
 6.255726683736575,
 8.330828826480984,
 4.017498171959036,
 6.493687771918642,
 5.725607874778171,
 6.786738252629342,
 8.452820870195385,
 0.30471330390680273,
 21.691135219941057,
 7.863636045340765,
 5.148413360501006,
 9.121334659543752,
 9.842908824573843,
 

In [22]:
sololongdf = pd.DataFrame( {"longsolo":sololong}, index = tickers)

In [23]:
merged_df = sololongdf.merge(profits, left_index=True, right_index=True)

In [24]:
merged_df.sort_values(by="longsolo", ascending= False)

,longsolo,profit
TSLA,101.213875,30.966629
NVDA,59.625743,47.337730
DXCM,54.869155,25.346031
AXON,47.118280,5.714846
NFLX,41.986912,29.981966
...,...,...
VTRS,0.481838,0.500622
PCG,0.387416,0.251056
DISH,0.313975,0.732350
APA,0.304713,0.050377


In [25]:
merged_df.to_excel("marceexcel.xlsx")

In [26]:
merged_df

,longsolo,profit
MMM,1.226692,1.482391
AOS,9.305986,3.685766
ABT,4.242648,1.437848
ABBV,4.195900,1.564586
ACN,6.334680,2.425557
...,...,...
YUM,5.403851,2.060100
ZBRA,9.243111,4.932026
ZBH,2.344700,1.054671
ZION,1.495124,0.735986


In [30]:

def vectorized ( df, slow, fast):
    ma_cal (df, slow, fast)
    first_buy = pd.Series (df.index == (df.slow < df.fast).idxmax(), index=df.index)
    
    real_signal = first_buy | (df.slow < df.fast).diff()
    trades = df[real_signal]
    if len (trades) % 2 !=0:
        mtm = df.tail(1).copy()
        mtm.price = mtm.Close
        trades =pd.concat([trades,mtm])
    profits = trades.price.diff()[1::2]/ trades.price[0::2].values
    gain = (profits + 1).prod()
    return gain

In [31]:
vectorized ( AAPL, 100,50)

9.728320042662135

In [32]:
results = []

for sym in tickers:
    subdf = slice_df (sym)
    print("result for" + sym)
    print(vectorized (subdf, 100,50))
    results.append(vectorized (subdf, 100,50))

result forMMM
1.4823914512417231
result forAOS
4.519996117320708
result forABT
1.4378484404677188
result forABBV
1.4983911697367178
result forACN
2.4255573838723232
result forATVI
2.417491290609178
result forADM
1.3738949353899947
result forADBE
5.132243964627093
result forADP
1.320237255653171
result forAAP
2.2492246926368455
result forAES
1.9034151147847516
result forAFL
2.701070451468886
result forA
2.3682717109492812
result forAPD
1.6832109409363807
result forAKAM
1.0413048718858373
result forALK
3.2438669894664094
result forALB
2.7479769680492288
result forARE
0.6044661713879131
result forALGN
5.451445737336812
result forALLE
0.6542165504002196
result forLNT
1.431672133172476
result forALL
2.29031695176343
result forGOOGL
2.8101830113948374
result forGOOG
2.95335499337628
result forMO
0.7679257207955459
result forAMZN
5.472995885750352
result forAMCR
0.5734304066700263
result forAMD
10.281675304206694
result forAEE
1.41883064971626
result forAAL
2.1702767892804284
result forAEP
0.